# Figure 5 Reconstruction

In [ ]:
# import requisite packages
import mne
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import sys,os
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,corrmap)
from mne.datasets import fetch_fsaverage
import os.path as op
from mne.datasets import eegbci
from mne.minimum_norm import make_inverse_operator, apply_inverse, apply_inverse_epochs 
import pickle
import pandas as pd

In [ ]:
# pull specific source solution and set up constants
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)
subject = 'fsaverage' # naming conventions for MNE
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation

In [ ]:
# define function to pull sources
def getsrc(name,alg):   
    cov = mne.read_cov(name+'-cov.fif');
    erp=mne.read_evokeds(name+'-ave.fif')[0];
    fwd=mne.read_forward_solution(name+'-fwd.fif');
    inv = make_inverse_operator(erp.info, fwd, cov, loose=0., depth=0.8,verbose=False);
    snr = 3.0
    lambda2 = 1.0 / snr ** 2
    # stc = abs(apply_inverse(erp, inv, lambda2, alg, verbose=False));
    stc = apply_inverse(erp, inv, lambda2, alg, verbose=False)
    src = inv["src"]
    return stc, src, inv, erp

In [ ]:
df = pd.read_csv('temp.csv')
df=df.loc[df.dome==1,:]

In [ ]:
df=df.dropna()
examp=df.savename

In [ ]:
df_fig=pd.DataFrame(data=None,index=examp,columns=['ActFM','ActCB','LEFT SIDE','Class','Type_Real'])

for idx in df_fig.index:
    df_fig.loc[idx,:]=df.loc[df.savename==idx,df_fig.columns.to_list()].values

In [ ]:
df_fig

In [ ]:
import time

In [ ]:
### this is for abs
# try to do some sort of averaging
# df_both=df_fig.loc[(df_fig.Class=='Both') & (df_fig.Type_Real=='EP1') & (df_fig['LEFT SIDE']==1),:]
# df_both=df_fig.loc[(df_fig.Class=='Both') & (df_fig.Type_Real=='EP2'),:]
# df_both=df_fig.loc[(df_fig.Class=='CB Only'),:]
df_both=df_fig.loc[(df_fig.Class=='Both') & (df_fig.Type_Real=='EP1'),:]

In [ ]:
fig,ax = plt.subplots(figsize=(5,5))
ax.plot(df_both.ActCB,df_both.ActFM,'ob')
ax.set_xlabel('% Activation CB')
ax.set_ylabel('% Activation FM')
ax.set_xlim(0,45)
ax.set_ylim(0,45)
ax.plot([0,45],[0,45],'--',c='gray')

ax.set_aspect('equal')

# fig.savefig("fig6_ex/pct_pct.pdf", bbox_inches="tight",dpi=600)

In [ ]:
eeg_idx=np.zeros(256)
for i in np.arange(0,len(eeg_idx),2):
    eeg_idx[i]=int(i+1)
    eeg_idx[i+1]=int(i)

eeg_idx = np.append(eeg_idx,[256])

In [ ]:

# stc_avg=np.zeros((20484, 501))
for i,examp in enumerate(df_both.index): # df_both.index: #
    stc, src, inv, erp = getsrc(examp,'sLORETA')
    # left side on top of right in the stc dataframe
    # np.concatenate((stc.lh_data,stc.rh_data),axis=0)
    if i==0:
        stc_base=stc;
        absmax=np.max([np.abs(stc.data.min()), stc.data.max()])
        
        erp_base=erp;
        erpmax=np.max([np.abs(erp.data.min()), erp.data.max()])
        
        if df_both.loc[examp,'LEFT SIDE']==1:
            stc_base.data = np.concatenate((stc.lh_data,stc.rh_data),axis=0)/absmax
            erp_base.data = erp.data/erpmax
        else:
            stc_base.data = np.concatenate((stc.rh_data,stc.lh_data),axis=0)/absmax
            erp_base.data = erp.data[eeg_idx.astype(int),:]/erpmax
    else:
        # stc_base.data=stc_base.data+stc.data 
        absmax=np.max([np.abs(stc.data.min()), stc.data.max()])
        erpmax=np.max([np.abs(erp.data.min()), erp.data.max()])
        if df_both.loc[examp,'LEFT SIDE']==1:
            stc_base.data = stc_base.data + np.concatenate((stc.lh_data,stc.rh_data),axis=0)/absmax
            erp_base.data = erp_base.data + erp.data/erpmax
        else:
            stc_base.data = stc_base.data + np.concatenate((stc.rh_data,stc.lh_data),axis=0)/absmax
            erp_base.data = erp_base.data + erp.data[eeg_idx.astype(int),:]/erpmax
    
stc_base.data=stc_base.data/len(df_both)
erp_base.data=erp_base.data/len(df_both)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

from matplotlib import cm
import matplotlib.cbook as cbook
import matplotlib.colors as colors

In [ ]:
# tpoints=[0.030,0.060];
tpoints=[0.060]
stc = stc_base;
# maxeeg=np.abs(erp.data[:,int(timepoint*1000)+100]).max()*1000000*1.1

# erp.plot_topomap(timepoint,colorbar=False,show=False,cmap='seismic',vlim=(-maxeeg,maxeeg))

for timepoint in tpoints:
    # nomin='mean_amp_CBOnly_Symmetric_n_Asymmetric_time({}ms)'.format(int(1000*timepoint))
    nomin='mean_amp_BOTH_Asymmetric_time({}ms)'.format(int(1000*timepoint))

    maxval=stc.data[:,100+int(1000*timepoint)].max()
    minval=stc.data[:,100+int(1000*timepoint)].min()
    
    absmax=np.max([maxval,np.abs(minval)])
    # absmax=1;

    cmap = cm.seismic
    
    # cnorm = matplotlib.colors.LogNorm(vmin=-absmax, vmax=absmax)
    # cnorm = matplotlib.colors.CenteredNorm(vcenter=0, halfrange=absmax)
    # cnorm = matplotlib.colors.SymLogNorm(linthresh=absmax*0.45, vmin=-absmax,vmax=absmax)
    cnorm = matplotlib.colors.SymLogNorm(linthresh=absmax*0.01, linscale=9,vmin=-absmax*0.5,vmax=absmax*0.5)
    smap = matplotlib.cm.ScalarMappable(norm=cnorm, cmap=cmap)
    
    view=['dorsal','ventral','lat','med','lat','med']
    hemi=['both','both','lh','lh','rh','rh']
    
    fig, ax = plt.subplot_mosaic(
        [[0,2,4,"cbar_stc"],
         [1,3,5,"cbar_stc"]],
        layout="constrained",
        figsize=(8,4),
        width_ratios=[1,1,1,0.1]
    )
    axs=[ax[0],ax[1],ax[2],ax[3],ax[4],ax[5],]

    
    for i in range(len(hemi)):
        brain=stc.plot(views=view[i],
                       hemi=hemi[i],
                       size=(500, 500),
                       subject="fsaverage",
                       subjects_dir=subjects_dir,
                       initial_time=timepoint,
                       time_viewer=False,
                       show_traces=False,
                       colormap=cmap,
                       cortex="low_contrast",
                       surface="pial",#pial, flat, white
                       alpha=0.7,
                       smoothing_steps=7,
                       background="white",
                       clim=dict(kind='value', lims=[-absmax, 0, absmax]),
                       colorbar=False,
                       )
        screenshot = brain.screenshot()
        time.sleep(0.1)
        brain.close()
        axs[i].imshow(screenshot)
        axs[i].axis('off');
    
    # cmap = plt.get_cmap('seismic')

    # import matplotlib
    # norm = matplotlib.colors.Normalize(vmin=-absmax, vmax=absmax)
    # mappable = matplotlib.cm.ScalarMappable(cmap=cmap,norm=norm)
    cbar = matplotlib.pyplot.colorbar(mappable = smap, cax=ax["cbar_stc"],label="Activation (AU)")

    plt.suptitle(nomin)
    plt.show(True)
    fig.savefig("fig6_ex/"+nomin+"_log.pdf", bbox_inches="tight",dpi=600)


        

In [ ]:
df_both=df_fig.loc[(df_fig.Class=='Both') & (df_fig.Type_Real=='EP1'),:]

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

from matplotlib import cm
import matplotlib.cbook as cbook
import matplotlib.colors as colors


# tpoints=[0.030,0.060];
tpoints=[0.060];
for examp in df_both.index[:]: # df_both.index: #
    stc, src, inv, erp = getsrc(examp,'sLORETA')
    for timepoint in tpoints:
        nomin='summary_{}_FM({})_CB({})_Symm({})_time({}ms)'.format(examp,df_both.loc[examp,'ActFM'],df_both.loc[examp,'ActCB'],df_both.loc[examp,'Type_Real'],int(1000*timepoint))

        maxval=stc.data[:,100+int(1000*timepoint)].max()
        minval=stc.data[:,100+int(1000*timepoint)].min()
        
        absmax=np.max([maxval,np.abs(minval)])
        # absmax=1;
    
        cmap = cm.seismic
        
        # cnorm = matplotlib.colors.LogNorm(vmin=-absmax, vmax=absmax)
        # cnorm = matplotlib.colors.CenteredNorm(vcenter=0, halfrange=absmax)
        cnorm = matplotlib.colors.SymLogNorm(linthresh=absmax*0.01, linscale=9,vmin=-absmax*0.5,vmax=absmax*0.5)
        smap = matplotlib.cm.ScalarMappable(norm=cnorm, cmap=cmap)
        
        view=['dorsal','ventral','lat','med','lat','med']
        hemi=['both','both','lh','lh','rh','rh']
        
        fig, ax = plt.subplot_mosaic(
            [["topo","cbar_topo", 0,2,4,"cbar_stc"],
             ["topo","cbar_topo", 1,3,5,"cbar_stc"]],
            layout="constrained",
            figsize=(10,4),
            width_ratios=[1,0.1,1,1,1,0.1]
        )
        axs=[ax[0],ax[1],ax[2],ax[3],ax[4],ax[5],]
        
        maxeeg=np.abs(erp.data[:,int(timepoint*1000)+100]).max()*1000000*1.1
        
        for i in range(len(hemi)):
            brain=stc.plot(views=view[i],
                    hemi=hemi[i],
                    size=(500, 500),
                    subject="fsaverage",
                    subjects_dir=subjects_dir,
                    initial_time=timepoint,
                    time_viewer=False,
                    show_traces=False,
                    colormap=cmap,
                    cortex="low_contrast",
                    surface="pial",#pial, flat, white
                    alpha=0.7,
                    smoothing_steps=7,
                    background="white",
                    clim=dict(kind='value', lims=[-absmax, 0, absmax]),
                    colorbar=False,
                    # clim=dict(kind='percent', lims=[90, 95, 100]),
                    # clim=dict(kind='value', lims=[0.1*maxval, 0.5*maxval, maxval]),
                      )
            screenshot = brain.screenshot()
            time.sleep(0.1)
            brain.close()
            axs[i].imshow(screenshot)
            axs[i].axis('off');
        
        # imago=erp.plot_topomap(timepoint,axes=ax["topo"],colorbar=False,show=False,cmap='seismic')
        
        cmap = plt.get_cmap('seismic')

        import matplotlib
        # norm = matplotlib.colors.Normalize(vmin=-absmax, vmax=absmax)
        # mappable = matplotlib.cm.ScalarMappable(cmap=cmap,norm=norm)
        # cbar = matplotlib.pyplot.colorbar(mappable = mappable, cax=ax["cbar_stc"],label="Activation (A/m^2)")
        cbar = matplotlib.pyplot.colorbar(mappable = smap, cax=ax["cbar_stc"],label="Activation (AU)")
        
        ladida=erp.plot_topomap(timepoint,axes=ax["topo"],colorbar=False,show=False,cmap='seismic')
        la=ladida.get_axes()
        imgs=la[0].get_images()
        vmin,vmax = imgs[0].get_clim()
        norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
        mappable = matplotlib.cm.ScalarMappable(cmap=cmap,norm=norm)
        cbar = matplotlib.pyplot.colorbar(mappable = mappable, cax=ax["cbar_topo"],label="Voltage (uV)")
        
        # erp.plot_topomap(timepoint,axes=(ax["topo"],ax['cbar_topo']),colorbar=True,cmap='seismic',vlim=(-maxeeg,maxeeg))
        # erp.plot_topomap(timepoint,axes=(ax["topo"],ax['cbar_topo']),colorbar=True,cmap='seismic')

        plt.suptitle(nomin)
        plt.show(True)
        # fdsa
        fig.savefig("fig6_ex/"+nomin+"_log_forsupp.pdf", bbox_inches="tight",dpi=600)
        # fdsa
    #print('summary_{}_FM({})_CB({})_Symm({})'.format(examp,df_both.loc[examp,'ActFM'],df_both.loc[examp,'ActCB'],df_both.loc[examp,'Type_Real']))